<a href="https://colab.research.google.com/github/tusharpoddar/e-Science-Research/blob/master/GoogleDrive/trainingVGGISH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The following notebook is used to train the VGG-model with the netcdf files that have been generated.

The notebook does the following things -
1. Used the data in the numpy format(creating it once again during training takes a lot of time).
2. Training the whole data of 1700 data points with single batch training for 10 times.
3. The notebook then plots the loss(cross entropy) after each epoch. 

## Data that is being used in the notebook -
The notebook loads two numpy files that contain the spectrogram data and the labels. This data has 1700 data points in total. 
1. Training Data - this set contins 1600 data points(randomly chosen). These are used for the training.

## Next step - 
The notebook does not test the performace of the model. So this can be seen as a dummy example of how to run the training of the model after passing. it some data. Proper validation and evaluation of the performance of the model is done in the next notebook called Single Prediction training. 

Note the label [1, 0] represents mice noise and [0, 1] represents random noise.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive' , force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!pip install numpy resampy tensorflow==1.15 tf_slim six soundfile

     |████████████████████████████████| 412.3MB 36kB/s 
     |████████████████████████████████| 358kB 37.4MB/s 
     |████████████████████████████████| 512kB 48.3MB/s 
     |████████████████████████████████| 3.8MB 47.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=707288f3189d53841c74d670431e62ed2b3d5819311e763fd5d0a5642fb6cedd
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfull

In [ ]:
pip install netcdf4

     |████████████████████████████████| 4.1MB 8.7MB/s 
     |████████████████████████████████| 286kB 40.5MB/s 


In [ ]:
# to open the selection tables
import pandas as pd
import numpy as np
import xarray as xr
import cv2

In [ ]:
path = '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/feature_data_frames/Old/annot_8features_100noise_fear.csv'

In [ ]:
annot = pd.read_csv(path)

In [ ]:
annot = pd.DataFrame(data = annot)
print(annot)
print(annot.Annotation.value_counts())

      Unnamed: 0  animal_number   session  ...  spec_kurt spec_slope     spec_roll
0              0            533   CPApair  ...  26.730238   0.000149  38781.012731
1              1            533   CPApair  ...  50.458618  -0.000471  37262.344520
2              2            533   CPApair  ...  35.797254  -0.001451  14898.363521
3              3            533   CPApair  ...  25.973715  -0.000857  24369.181905
4              4            533   CPApair  ...   4.124133  -0.001831  19606.292614
...          ...            ...       ...  ...        ...        ...           ...
1754        1595            557  cagepair  ...   0.945694  -0.000005  26382.647835
1755        1596            557  cagepair  ...   1.348464  -0.000287  24299.949586
1756        1597            557  cagepair  ...   5.296944  -0.001228  19409.636199
1757        1598            557  cagepair  ...   0.873569   0.000367  29020.685860
1758        1599            557  cagepair  ...   1.744215  -0.000575  26036.829758

[17

In [ ]:
annot_np = np.array(annot)

In [ ]:
# opeing a data file to check if the time stamps exists or not 
path = '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/netcdf_files/Old/Fear/533_CPApair_xr_Dataset.nc'
data = xr.open_dataset(path)

In [ ]:
# function to produce the path of the netcdf file that is specified in the selection table
def pathGen(animalNumber, experimentName):
  return '/content/gdrive/Shared drives/USV_eScience_Incubator/Data/netcdf_files/Old/Fear/' + str(animalNumber)+'_' +str(experimentName) + '_xr_Dataset.nc'

In [ ]:
# the following function uses the selection table to generate 2 arrays - 
# the numpy array that represents the spectrogram information and
# the other that helps us to differentiate between the mice noise and random noise
def prepare_data(annot_np):
  dataArray = []
  labelArray = []
  for record in annot_np:
    animalNumber = record[1]
    experimentName = record[2]
    time = record[3]
    path = pathGen(animalNumber, experimentName)
    print(path)
    data = xr.open_dataset(path)
  
    print(time)
    xr_slice = data['__xarray_dataarray_variable__'].sel(slices=time)
    temp = np.array(xr_slice)
    # in order to change the shape of the spectrogram so that it matches the format meeded for by the vggish model
    changed = cv2.resize(temp, (64, 96))
    dataArray.append(changed)
    if record[4] == 'rand_noise':
      labelArray.append([0, 1])
    else:
      labelArray.append([1, 0])

  # once both the arrays are formed i need to convert them to numpy array and then return them
  numpyData = np.array(dataArray)
  numpyLabel = np.array(labelArray)
  return numpyData, numpyLabel

In [ ]:
cd /content/gdrive/Shared drives/Research

/content/gdrive/Shared drives/Research


In [ ]:
from __future__ import print_function

from random import shuffle

import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tf_slim as slim

import vggish_input
import vggish_params
import vggish_slim

flags = tf.app.flags

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
flags = tf.app.flags

flags.DEFINE_integer(
    'num_batches', 10,
    'Number of batches of examples to feed into the model. Each batch is of '
    'variable size and contains shuffled examples of each class of audio.')

flags.DEFINE_boolean(
    'train_vggish', True,
    'If True, allow VGGish parameters to change during training, thus '
    'fine-tuning VGGish. If False, VGGish parameters are fixed, thus using '
    'VGGish as a fixed feature extractor.')

flags.DEFINE_string(
    'checkpoint', 'vggish_model.ckpt',
    'Path to the VGGish checkpoint file.')

FLAGS = flags.FLAGS

_NUM_CLASSES = 2

In [ ]:
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [ ]:
def _get_examples_batch(numpyData, numpyLabel):
  """Returns a shuffled batch of examples of all audio classes.

  Note that this is just a toy function because this is a simple demo intended
  to illustrate how the training code might work.

  Returns:
    a tuple (features, labels) where features is a NumPy array of shape
    [batch_size, num_frames, num_bands] where the batch_size is variable and
    each row is a log mel spectrogram patch of shape [num_frames, num_bands]
    suitable for feeding VGGish, while labels is a NumPy array of shape
    [batch_size, num_classes] where each row is a multi-hot label vector that
    provides the labels for corresponding rows in features.
  """
  # Shuffle (example, label) pairs across all classes.
  labeled_examples = list(zip(numpyData, numpyLabel))
  shuffle(labeled_examples)

  # Separate and return the features and labels.
  features = [example for (example, _) in labeled_examples]
  labels = [label for (_, label) in labeled_examples]
  return (features, labels)

In [ ]:
with tf.Graph().as_default(), tf.Session() as sess:
    # Define VGGish.
    embeddings = vggish_slim.define_vggish_slim(FLAGS.train_vggish)

    # Define a shallow classification model and associated training ops on top
    # of VGGish.
    with tf.variable_scope('mymodel'):
      # Add a fully connected layer with 100 units.
      num_units = 100
      fc = slim.fully_connected(embeddings, num_units)

      # Add a classifier layer at the end, consisting of parallel logistic
      # classifiers, one per class. This allows for multi-class tasks.
      logits = slim.fully_connected(
          fc, _NUM_CLASSES, activation_fn=None, scope='logits')
      tf.sigmoid(logits, name='prediction')

      # Add training ops.
      with tf.variable_scope('train'):
        global_step = tf.Variable(
            0, name='global_step', trainable=False,
            collections=[tf.GraphKeys.GLOBAL_VARIABLES,
                         tf.GraphKeys.GLOBAL_STEP])

        # Labels are assumed to be fed as a batch multi-hot vectors, with
        # a 1 in the position of each positive class label, and 0 elsewhere.
        labels = tf.placeholder(
            tf.float32, shape=(None, _NUM_CLASSES), name='labels')

        # Cross-entropy label loss.
        xent = tf.nn.sigmoid_cross_entropy_with_logits(
            logits=logits, labels=labels, name='xent')
        loss = tf.reduce_mean(xent, name='loss_op')
        tf.summary.scalar('loss', loss)

        # We use the same optimizer and hyperparameters as used to train VGGish.
        optimizer = tf.train.AdamOptimizer(
            learning_rate=vggish_params.LEARNING_RATE,
            epsilon=vggish_params.ADAM_EPSILON)
        optimizer.minimize(loss, global_step=global_step, name='train_op')

    # Initialize all variables in the model, and then load the pre-trained
    # VGGish checkpoint.
    sess.run(tf.global_variables_initializer())
    vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)

    # Locate all the tensors and ops we need for the training loop.
    features_tensor = sess.graph.get_tensor_by_name(
        vggish_params.INPUT_TENSOR_NAME)
    labels_tensor = sess.graph.get_tensor_by_name('mymodel/train/labels:0')
    global_step_tensor = sess.graph.get_tensor_by_name(
        'mymodel/train/global_step:0')
    loss_tensor = sess.graph.get_tensor_by_name('mymodel/train/loss_op:0')
    train_op = sess.graph.get_operation_by_name('mymodel/train/train_op')
    # calculating the numpy array only once
    
    #numpyData, numpyLabel = prepare_data(annot_np)
    
    numpyData = np.load('/content/gdrive/Shared drives/Research/Data/Numpy files/annot_df_CPApost_round2/fearNoiseSpectrogram.npy')
    numpyLabel = np.load('/content/gdrive/Shared drives/Research/Data/Numpy files/annot_df_CPApost_round2/fearNoiseLabel.npy')
    # The training loop.
    for _ in range(FLAGS.num_batches):
      (features, labels) = _get_examples_batch(numpyData, numpyLabel)
      [num_steps, loss, _] = sess.run(
          [global_step_tensor, loss_tensor, train_op],
          feed_dict={features_tensor: features, labels_tensor: labels})
      print('Step %d: loss %g' % (num_steps, loss))

Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Restoring parameters from vggish_model.ckpt
Step 1: loss 2.91402
Step 2: loss 2.3604
Step 3: loss 1.91897
Step 4: loss 1.57996
Step 5: loss 1.32861
Step 6: loss 1.15649
Step 7: loss 1.04067
Step 8: loss 0.949387
Step 9: loss 0.871084
Step 10: loss 0.807416
